In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers, regularizers

# Load train and test datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Checking information of the train data
train_df.info()

# Initialize LabelEncoder and dictionary to store encoders for each column
label_encoders = {}

# List of categorical columns to encode
categorical_columns = ['model', 'motor_type', 'wheel', 'color', 'status', 'type']

# Apply LabelEncoder to each categorical column for both train and test
for col in categorical_columns:
    label_encoders[col] = LabelEncoder()
    train_df[col] = label_encoders[col].fit_transform(train_df[col])
    test_df[col] = test_df[col].apply(lambda x: label_encoders[col].transform([x])[0] if x in label_encoders[col].classes_ else -1)

# Function to convert running values from 'km' to 'miles'
def convert_running(value):
    if 'km' in value:
        kilometers = int(value.split()[0])
        miles = kilometers * 0.621371
        return f"{miles:.2f} miles"
    elif 'miles' in value:
        return value
    else:
        return value 

# Apply the conversion for both train and test datasets
train_df['running'] = train_df['running'].apply(convert_running)
test_df['running'] = test_df['running'].apply(convert_running)

# Clean and convert 'running' column to numeric values
train_df['running'] = train_df['running'].str.replace('miles', '').str.strip()
test_df['running'] = test_df['running'].str.replace('miles', '').str.strip()
train_df['running'] = pd.to_numeric(train_df['running'], errors='coerce').fillna(0).astype('int64')
test_df['running'] = pd.to_numeric(test_df['running'], errors='coerce').fillna(0).astype('int64')

# Drop duplicates from the training data
train_df = train_df.drop_duplicates()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1642 entries, 0 to 1641
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         1642 non-null   object 
 1   year          1642 non-null   int64  
 2   motor_type    1642 non-null   object 
 3   running       1642 non-null   object 
 4   wheel         1642 non-null   object 
 5   color         1642 non-null   object 
 6   type          1642 non-null   object 
 7   status        1642 non-null   object 
 8   motor_volume  1642 non-null   float64
 9   price         1642 non-null   int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 128.4+ KB


In [6]:

# Prepare features and target variable
X = train_df.drop(['price', 'wheel'], axis = 1) # Replace 'target_column' with your actual target column name
y = train_df['price']  # Your target variable

# # Split the data into training and validation sets
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)




In [8]:
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X)
# X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(test_df.drop(columns=['Id', 'wheel']))  # Adjust as necessary


In [10]:

# Define the deep learning model with improved architecture
model = keras.Sequential([
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01), input_shape=(X_train_scaled.shape[1],)),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(1)  # Output layer for regression
])


C:\Users\admin\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:

# Compile the model with a learning rate scheduler
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_absolute_error', metrics=['mae'])


In [14]:

# Fit the model with early stopping
early_stopping = keras.callbacks.EarlyStopping(monitor='val_mae', patience=10, restore_best_weights=True)


In [16]:

history = model.fit(X_train_scaled, y, validation_split=0.2, 
                    epochs=200, batch_size=32, callbacks=[early_stopping], verbose=1)



Epoch 1/200
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 15413.9453 - mae: 15411.4199 - val_loss: 16623.8359 - val_mae: 16621.4102
Epoch 2/200
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 15785.1152 - mae: 15782.3408 - val_loss: 16172.5166 - val_mae: 16167.7705
Epoch 3/200
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 14821.8789 - mae: 14815.8291 - val_loss: 13586.0000 - val_mae: 13574.7969
Epoch 4/200
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 11400.2773 - mae: 11386.7734 - val_loss: 6421.3325 - val_mae: 6400.2085
Epoch 5/200
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5119.8149 - mae: 5096.7031 - val_loss: 3691.1184 - val_mae: 3666.5325
Epoch 6/200
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3957.5474 - mae: 3933.0867 - val_loss: 3405.5410 - val_mae: 3380.7622
Epoch 7/200
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3534.3635 - mae: 3509.6794 - val_loss: 3238.3391 - val_mae: 3213.6201
Epoch 8/200
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3527.0085 - mae: 3502.31

In [20]:
# Evaluate the model on the validation set
val_loss, val_mae = model.evaluate(X_train_scaled, y)
print(f'Validation Mean Absolute Error: {val_mae:.2f}')

52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2813.0249 - mae: 2789.6826
Validation Mean Absolute Error: 2820.52


In [41]:

# Make predictions on the test set
y_predict = model.predict(X_test_scaled)



13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


In [45]:
# Create a DataFrame for the predictions
df_predictions = pd.DataFrame(y_predict, columns=['price'])

# Display predictions
print(df_predictions)



            price
0    19169.154297
1    16620.316406
2    18832.302734
3    15628.364258
4     7077.588379
..            ...
406  22526.234375
407  13498.885742
408   9379.782227
409  17515.326172
410  12814.867188

[411 rows x 1 columns]


In [47]:
combined_df = pd.concat([test_df['Id'], df_predictions], axis=1)

combined_df

,Id,price
0,0,19169.154297
1,1,16620.316406
2,2,18832.302734
3,3,15628.364258
4,4,7077.588379
...,...,...
406,406,22526.234375
407,407,13498.885742
408,408,9379.782227
409,409,17515.326172


In [49]:
combined_df.to_csv('Aba.csv', index=False)